# Ciclo 02: Aprendizado Supervisionado - Classificação

## 1.0 Import Libraries

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt

## 1.1 Load Dataset

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar


Esse dataset foi usado no último hackday. Nossa tarefa era treinar algoritmos para prever se o banco deveria liberar um empréstimo para os seus clientes, ao final do treinamento nós comparamos os outputs com targets da coluna `limite_adicional`.

In [6]:
df['limite_adicional'].unique()

array(['Negar', 'Conceder'], dtype=object)

In [7]:
df['limite_adicional'].value_counts()

Negar       7995
Conceder    1505
Name: limite_adicional, dtype: int64

## 2.0 Seleção de features

In [8]:
df.columns

Index(['id_cliente', 'idade', 'saldo_atual', 'divida_atual', 'renda_anual',
       'valor_em_investimentos', 'taxa_utilizacao_credito', 'num_emprestimos',
       'num_contas_bancarias', 'num_cartoes_credito', 'dias_atraso_dt_venc',
       'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros',
       'investe_exterior', 'pessoa_polit_exp', 'limite_adicional'],
      dtype='object')

In [9]:
# x_train = dataset que contém as features de treinamento
# y_train = series que contém os labels (rótulos das classes)

features = ['idade', 'saldo_atual', 'divida_atual', 'renda_anual',
       'valor_em_investimentos', 'taxa_utilizacao_credito', 'num_emprestimos',
       'num_contas_bancarias', 'num_cartoes_credito', 'dias_atraso_dt_venc',
       'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros']

label = 'limite_adicional'


x_train = df.loc[:, features]
y_train = df.loc[:, label]

## 3.0 Treinamento

In [10]:
# definição parâmetro de treinamento
k = 7

# criação do objeto que perfomará o algoritmo de classificação. Nesse caso
# o objeto 'knn_classifier' é uma instância da classe 'KNeighborsClassifier'
# que avaliará treinará o algoritmo utilizando os 7 vizinhos mais próximos.
knn_classifier = KNeighborsClassifier(n_neighbors=k)

In [11]:
type(knn_classifier)

sklearn.neighbors._classification.KNeighborsClassifier

In [12]:
knn_classifier

KNeighborsClassifier(n_neighbors=7)

Vamos agora treinar o algoritmo no conjunto de treino `x_train`. Para isso nós chamamos o método `.fit()` do objeto `KNeighborsClassifier`e passamos como parâmetros o conjunto de treino e os rótulos das classes.

In [13]:
# treinamento do algoritmo no conjunto de dados.
knn_classifier.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=7)

Vamos agora utilizar o algoritmo treinado para tentar prever qual é classe (`Liberar` ou `Negar`) de cada registro (cliente) do dataset de treino. Para isso nós chamamos o método `.predict()`.

In [14]:
y_pred = knn_classifier.predict(x_train)

In [15]:
# esses são os rótulos atribuídos para os clientes pelo algoritmo
y_pred

array(['Negar', 'Negar', 'Negar', ..., 'Negar', 'Negar', 'Negar'],
      dtype=object)

Para visualizarmos melhor, vamos criar um dataframe com os rótulos originais e os atribuídos pelo algoritmo.

In [19]:
df_result = df.copy()
df_result['classificacao'] = y_pred

df_result.loc[:, ['id_cliente', 'idade', 'limite_adicional', 'classificacao']].sample(10)

,id_cliente,idade,limite_adicional,classificacao
2155,10385,39,Negar,Negar
4956,8261,35,Negar,Negar
3753,1544,34,Negar,Negar
6795,1472,25,Negar,Negar
8013,10602,24,Negar,Negar
7192,4349,25,Negar,Negar
7861,4862,35,Negar,Negar
8138,1858,22,Negar,Conceder
469,10985,43,Negar,Negar
4494,7531,47,Negar,Negar


## 4.0 Performance

### Matriz de Confusão e Acurácia

Antes de verificarmos a performance do nosso algoritmo, vamos primeiro verificar o balanceamento do dataset.

In [20]:
df.loc[:, 'limite_adicional'].value_counts()

Negar       7995
Conceder    1505
Name: limite_adicional, dtype: int64

In [21]:
df.loc[:, 'limite_adicional'].value_counts(normalize=True)

Negar       0.841579
Conceder    0.158421
Name: limite_adicional, dtype: float64

Uma maneira não muito convencional de avaliarmos a performance do nosso algoritmo seria nos criamos uma nova coluna chamada `acertos` com valor `1` se os valores da coluna `limite_adicional` forem iguais a `classificacao` e `0` caso contrário e depois somar os valores dessa coluna.

In [22]:
df_result['acertos'] = ( df_result.loc[:, ['id_cliente', 'limite_adicional', 'classificacao']]
                        .apply(lambda x: 1 if x['limite_adicional'] == x['classificacao'] else 0, axis=1) )

df_result.loc[:, ['id_cliente', 'idade', 'limite_adicional', 'classificacao', 'acertos']].sample(10)

,id_cliente,idade,limite_adicional,classificacao,acertos
6909,2254,26,Conceder,Conceder,1
4767,5764,42,Negar,Negar,1
8560,83,43,Negar,Negar,1
4883,10645,24,Conceder,Conceder,1
4578,2009,19,Negar,Negar,1
881,9298,24,Negar,Negar,1
7541,8929,53,Negar,Negar,1
4151,10021,506,Negar,Negar,1
7965,5519,24,Conceder,Negar,0
8291,6860,36,Negar,Negar,1


In [26]:
print(f"Acurácia: {df_result['acertos'].sum() / df_result.shape[0]:.5f}")

Acurácia: 0.85905


Uma maneira muito mais eficiente de fazermos a avaliação do módelo é utilizando o módulo `metrics` do pacote `sklearn`.

```python
from sklearn import metrics as mt
```

Para obtermos a matriz de confusão, fazemos uso da função `.confusion_matrix()`.

In [27]:
# para calcularmos a matriz de confusão, precisamos passar como parâmetros os rótulos reais e os previstos 
mt.confusion_matrix(y_train, y_pred)

array([[ 369, 1136],
       [ 203, 7792]])

Já para calcularmos a acurácia do algoritmo usamos a função `.accuracy_score()`

In [28]:
mt.accuracy_score(y_train, y_pred)

0.8590526315789474

### Precision e Recall

Para avaliarmos a precisão do nosso algoritmo usar a função `.precision_score()`. Nesse caso devemos passar um parâmetro extra (`pos_label`) para indicar qual é a classe positiva.

In [32]:
# estamos verificando qual a precisão da classe 'Negar'
mt.precision_score(y_train, y_pred, pos_label='Negar')

0.8727598566308243

In [33]:
# estamos verificando qual a precisão da classe 'Conceder'
mt.precision_score(y_train, y_pred, pos_label='Conceder')

0.6451048951048951

Para avaliarmos a precisão do nosso algoritmo usar a função `.recall_score()`. Também devemos passar o parâmetro `pos_label` para indicar qual é a classe positiva.

In [34]:
# estamos verificando qual o Recall da classe 'Negar'
mt.recall_score(y_train, y_pred, pos_label='Negar')

0.9746091307066916

In [35]:
# estamos verificando qual a Recall da classe 'Conceder'
mt.recall_score(y_train, y_pred, pos_label='Conceder')

0.24518272425249169